In [10]:
# --- CÉLULA 1: Setup e Carregamento de Bibliotecas ---
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Modelos para a etapa de Regressão Não Linear/Ensemble
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb # Ou lightgbm/catboost, vamos focar no XGBoost como principal

# Para garantir o uso correto do RMSE em todas as versões do sklearn
from math import sqrt 

import joblib
import os

print("Bibliotecas importadas e prontas para Regressão Não Linear e Ensembles.")

Bibliotecas importadas e prontas para Regressão Não Linear e Ensembles.


In [6]:
# --- CÉLULA 2: Carregamento, Pré-processamento e Salvamento de Dados ---
# 1. Carregamento e Definição de X e Y
housing = fetch_california_housing(as_frame=True)
housing_df = housing.frame
housing_df['MEDV'] = housing_df['MedHouseVal'] # Variável Alvo (Y)

X = housing_df.drop(['MEDV', 'MedHouseVal'], axis=1)
y = housing_df['MEDV']

# 2. Split (random_state=42 garante a mesma divisão sempre)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# 3. Padronização (Ajustar apenas no treino, transformar no treino e teste)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --------------------------------------------------------------------------
# 4. SALVAMENTO DOS DADOS PARA REPRODUTIBILIDADE
# --------------------------------------------------------------------------

DATA_DIR = 'data'

# 4.1. Garantir que o diretório exista
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR) 
    print(f"Diretório '{DATA_DIR}' criado.")

# 4.2. Salvar os conjuntos de dados processados
joblib.dump(X_train_scaled, os.path.join(DATA_DIR, 'X_train_scaled.joblib'))
joblib.dump(X_test_scaled, os.path.join(DATA_DIR, 'X_test_scaled.joblib'))
joblib.dump(y_train, os.path.join(DATA_DIR, 'y_train.joblib'))
joblib.dump(y_test, os.path.join(DATA_DIR, 'y_test.joblib'))

print(f"Dados padronizados (X/y) salvos com sucesso em: {DATA_DIR}/")

# Converte para DataFrame para uso em células futuras, se necessário (mantendo as colunas)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

print(f"\nDados do California Housing carregados, padronizados e separados (Train/Test).")
print(f"Shape do X_train: {X_train_scaled.shape}")

Diretório 'data' criado.
Dados padronizados (X/y) salvos com sucesso em: data/

Dados do California Housing carregados, padronizados e separados (Train/Test).
Shape do X_train: (14448, 8)


In [4]:
# --- CÉLULA 3: Regressão KNN (K-Nearest Neighbors) ---

# Importa módulos necessários para esta célula (alguns já devem estar na Célula 1)
from sklearn.neighbors import KNeighborsRegressor
import os 
import joblib 
from math import sqrt 

# 1. Definir o range de K's a serem testados
param_grid = {'n_neighbors': range(1, 21)} # Testar de K=1 a K=20

# 2. Configurar o GridSearchCV para encontrar o K ideal
knn = KNeighborsRegressor()
knn_grid = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    scoring='neg_mean_squared_error', # Otimizar para minimizar o MSE
    cv=5, # 5-fold cross-validation
    verbose=0,
    n_jobs=-1 # Usar todos os núcleos
)

# 3. Treinar com os dados padronizados
knn_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador encontrado (Organização e Reprodutibilidade)
best_knn_model = knn_grid.best_estimator_

# Lógica de Salvamento na pasta 'models/'
MODEL_DIR = 'models'
MODEL_FILENAME = 'knn_regressor_model.joblib'
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_FILENAME) # Caminho relativo

# 4.1. Garantir que o diretório exista
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR) # Cria o diretório 'models/' se ele não existir
    print(f"Diretório '{MODEL_DIR}' criado.")

# 4.2. Salvar o modelo
joblib.dump(best_knn_model, MODEL_PATH) 
print(f"Modelo KNN otimizado salvo em: {MODEL_PATH}")

# 5. Avaliação do Modelo Otimizado (Usando o modelo salvo)
y_pred_knn = best_knn_model.predict(X_test_scaled)
rmse_knn = sqrt(mean_squared_error(y_test, y_pred_knn)) 
r2_knn = r2_score(y_test, y_pred_knn)

print(f"\nKNN: Melhor K encontrado: {knn_grid.best_params_['n_neighbors']}")
print(f"RMSE (Teste) - KNN: {rmse_knn:.4f}")
print(f"R2 (Teste) - KNN: {r2_knn:.4f}")

Diretório 'models' criado.
Modelo KNN otimizado salvo em: models\knn_regressor_model.joblib

KNN: Melhor K encontrado: 11
RMSE (Teste) - KNN: 0.6411
R2 (Teste) - KNN: 0.6869


In [7]:
# --- CÉLULA 4: Regressão Random Forest (Ensemble) ---

# Random Forest não necessita de Padronização, mas funciona bem com ela.

# 1. Definir o range de hiperparâmetros (apenas uma pequena otimização inicial)
param_grid_rf = {
    'n_estimators': [100, 200], # Número de árvores
    'max_depth': [10, 20],      # Profundidade máxima da árvore
    'min_samples_split': [5, 10]
}

# 2. Configurar o GridSearchCV
rf = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_rf,
    scoring='neg_mean_squared_error',
    cv=3, # Usamos CV=3 aqui para acelerar o processo, já que são muitos dados.
    verbose=1,
    n_jobs=-1
)

# 3. Treinar
rf_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_rf_model = rf_grid.best_estimator_

MODEL_DIR = 'models'
MODEL_PATH_RF = os.path.join(MODEL_DIR, 'random_forest_regressor_model.joblib')
joblib.dump(best_rf_model, MODEL_PATH_RF)
print(f"\nModelo Random Forest otimizado e salvo em: {MODEL_PATH_RF}")

# 5. Avaliação
y_pred_rf = best_rf_model.predict(X_test_scaled)
rmse_rf = sqrt(mean_squared_error(y_test, y_pred_rf)) 
r2_rf = r2_score(y_test, y_pred_rf)

print(f"\nRandom Forest: Melhor Parâmetro encontrado: {rf_grid.best_params_}")
print(f"RMSE (Teste) - Random Forest: {rmse_rf:.4f}")
print(f"R2 (Teste) - Random Forest: {r2_rf:.4f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits

Modelo Random Forest otimizado e salvo em: models\random_forest_regressor_model.joblib

Random Forest: Melhor Parâmetro encontrado: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 200}
RMSE (Teste) - Random Forest: 0.5046
R2 (Teste) - Random Forest: 0.8060


In [8]:
# --- CÉLULA 5: Regressão XGBoost (Gradient Boosting) ---

# Reutilizamos o import do XGBoost feito na Célula 1 (se a instalação deu certo)
from xgboost import XGBRegressor 

# 1. Definir o range de hiperparâmetros (apenas uma pequena otimização inicial)
param_grid_xgb = {
    'n_estimators': [100, 200],      # Número de árvores
    'max_depth': [5, 10],            # Profundidade máxima
    'learning_rate': [0.05, 0.1]     # Taxa de aprendizado
}

# 2. Configurar o GridSearchCV
# Usamos o 'gbtree' como booster padrão para regressão
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_grid = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='neg_mean_squared_error',
    cv=3, # Usamos CV=3 para acelerar o processo
    verbose=1,
    n_jobs=-1
)

# 3. Treinar
# O XGBoost pode aceitar dados padronizados (scaled) ou brutos, usaremos o scaled para consistência.
xgb_grid.fit(X_train_scaled, y_train)

# 4. Salvar o melhor estimador
best_xgb_model = xgb_grid.best_estimator_

MODEL_DIR = 'models'
MODEL_PATH_XGB = os.path.join(MODEL_DIR, 'xgboost_regressor_model.joblib')
joblib.dump(best_xgb_model, MODEL_PATH_XGB)
print(f"\nModelo XGBoost otimizado e salvo em: {MODEL_PATH_XGB}")

# 5. Avaliação
y_pred_xgb = best_xgb_model.predict(X_test_scaled)
rmse_xgb = sqrt(mean_squared_error(y_test, y_pred_xgb)) 
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"\nXGBoost: Melhor Parâmetro encontrado: {xgb_grid.best_params_}")
print(f"RMSE (Teste) - XGBoost: {rmse_xgb:.4f}")
print(f"R2 (Teste) - XGBoost: {r2_xgb:.4f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits

Modelo XGBoost otimizado e salvo em: models\xgboost_regressor_model.joblib

XGBoost: Melhor Parâmetro encontrado: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
RMSE (Teste) - XGBoost: 0.4642
R2 (Teste) - XGBoost: 0.8358
